In [1]:
#Import the necessary libraries
import core.coreapi as coreapi
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook
import plotly.graph_objs as go
from matplotlib import colormaps
import datetime

In [2]:
#Instatiate the API class

qmapi = coreapi.QuantmatixAPI()

Working in environment: PROD


<h1> INSIDER EMPLOYEE TRADES <h1>

In [8]:
#First need to define the SQL string

#this specifically takes the trading done by employees of a company
sql_query = """
SELECT [TickerId]
      ,[FilingDate]
      ,[TransactionDate]
      ,[ReportingCik]
      ,[CompanyCik]
      ,[TransactionType]
      ,[SecuritiesOwned]
      ,[SecuritiesTransacted]
      ,[ReportingName]
      ,[TypeOfOwner]
      ,[Link]
      ,[SecurityName]
      ,[Price]
      ,[FormType]
      ,[AcquisitionOrDisposition]
  FROM [dbo].[TickerInsiderTradingFMP]
"""
sql_db_name = "STUDY_DATABASE_NAME"

In [9]:
#Read in the data
insider_data = qmapi.get_data_sql(sql_string=sql_query,
                                  db_name=sql_db_name)
insider_data.head()

,TickerId,FilingDate,TransactionDate,ReportingCik,CompanyCik,TransactionType,SecuritiesOwned,SecuritiesTransacted,ReportingName,TypeOfOwner,Link,SecurityName,Price,FormType,AcquisitionOrDisposition
0,12265,2003-05-05 10:58:35,2003-05-01,0001183377,0001144215,A-Award,3814.0,39.0,DEAVENPORT EARNEST W JR,director,https://www.sec.gov/Archives/edgar/data/118337...,Stock Units,0.000,4,A
1,12265,2003-05-05 11:12:49,2003-05-01,0001183380,0001144215,A-Award,3814.0,39.0,NORTH JULIA B,director,https://www.sec.gov/Archives/edgar/data/118338...,Stock Units,0.000,4,A
2,4641,2003-05-05 15:44:13,2003-05-02,0001199338,0000034088,M-Exempt,20000.0,16824.0,CRAMER HAROLD R,officer: Vice President,https://www.sec.gov/Archives/edgar/data/34088/...,Employee Stock Option (Right to Buy),15.280,4,D
3,13262,2003-05-05 16:39:32,2003-05-02,0001182403,0000833640,M-Exempt,270097.0,15000.0,EARHART HOWARD,director,https://www.sec.gov/Archives/edgar/data/118240...,Non-Qualified Stock Option(right to buy),12.100,4,D
4,4641,2003-05-05 16:45:07,2003-05-02,0001198809,0000034088,M-Exempt,700000.0,100000.0,RAYMOND LEE R,"director, officer: Chairman and President",https://www.sec.gov/Archives/edgar/data/34088/...,Employee Stock Option (Right to Buy),15.125,4,D


In [10]:
#first find the types of transactions in the dataset
trans_type = insider_data['TransactionType'].unique()
print(trans_type)

#we have many different types of transactions...

['A-Award' 'M-Exempt' 'P-Purchase' 'C-Conversion' 'X-InTheMoney' 'S-Sale'
 'J-Other' 'G-Gift' 'E-ExpireShort' 'I-Discretionary' 'D-Return'
 'H-ExpireLong' 'W-Will' 'F-InKind' 'Z-Trust' 'L-Small' 'U-Tender'
 'O-OutOfTheMoney']


In [11]:
#this returns the stocks with the most insider trades
insider_data['TickerId'].value_counts().head(20)

#lets look into the most inside traded stock, which today is 1910 = Salesforce (CRM)

TickerId
4545     7533
12780    6452
4195     6205
2687     5271
1333     5181
12618    5170
1434     5149
1268     5050
1940     5004
2737     4814
4678     4762
1692     4761
2862     4725
12465    4598
13229    4503
3407     4429
2542     4406
12528    4375
3589     4336
3630     4120
Name: count, dtype: int64

In [12]:
ticker_of_choice = 1414

####################################################################################################

#change dataset to have only the ticker of choice trades
ticker_df = insider_data[(insider_data['TickerId'] == ticker_of_choice)] #get all the trades associated with our stock of choice

#get data of ticker from API
ticker_of_choice_data = qmapi.get_tickers_data(tickers_list=[ticker_of_choice])

#read in stock ticker data from yahoo finance

#our start date is the first recorded trade for reading in Yahoo data
start_date=ticker_df['TransactionDate'].min()

#our end date is today
end_date=datetime.date.today()

#ticker short name got from API data
ticker = ticker_of_choice_data['shortName'].str.split(':').str[0]

#read in historical price data
ticker_data = yf.download(ticker[0], start=start_date, end=end_date, interval='1d')

########################################################################################################


#CREATE THE PLOT

highlight_dates  = ticker_df['TransactionDate'] #take dates of transactions
highlight_dates = pd.to_datetime(highlight_dates) #convert to datetime
transaction_type = ticker_df['TransactionType'] #take all transaction types associated with the dates

highlight_df = pd.DataFrame({'Date':highlight_dates, 'Type':transaction_type}) #make a new dataframe containing dates and types

#find the number of unique transaction types
unique_types = highlight_df['Type'].unique()
num_types = len(unique_types)

# Use a colormap to assign colors (using matplotlib for color generation)
colors = colormaps['Set1'](np.linspace(0, 1, num_types))  
color_map = {typ: f'rgb({int(c[0] * 255)}, {int(c[1] * 255)}, {int(c[2] * 255)})'
              for typ, c in zip(unique_types, colors)}  # Map each type to a color

def get_next_available_date(date, ticker_index):
    max_date = ticker_index.max()
    while date not in ticker_index and date <= max_date:
        date += pd.Timedelta(days=1)
    return date if date in ticker_index else None

#plot ticker and transaction dates
fig = go.Figure()

#plot ticker data
fig.add_trace(go.Scatter(x=ticker_data.index, 
                         y=ticker_data['Close'],
                         mode='lines',
                         name=f'{ticker[0]} Daily Close Price'))

#plot transaction dates and types
legend_added = set()

for _, row in highlight_df.iterrows():
    date = row['Date']
    type = row['Type']
    adjusted_date = date if date in ticker_data.index else get_next_available_date(date, ticker_data.index)
    if adjusted_date in ticker_data.index:
        fig.add_trace(go.Scatter(x = [date],
                                y = [ticker_data.loc[adjusted_date, 'Close']],
                                mode='markers',
                                name=type,
                                marker=dict(color=color_map[type], size=8),
                                showlegend=type not in legend_added,
                                legendgroup=type,
                                visible='legendonly'
                                ))
        legend_added.add(type)
    else:
        print(f"Date {date} not found in ticker_data index.") #sometimes the date is not in the ticker data, so this avoids an error
    

fig.update_layout(
    title = f'{ticker_of_choice_data['instrumentName'].iloc[0]} Historical Price & Insider Dealings',
    xaxis_title = 'Date',
    yaxis_title = 'Price',
    showlegend = True,
    legend = dict(groupclick="togglegroup")
)

fig.show()

[*********************100%%**********************]  1 of 1 completed


In [103]:
ticker_df[ticker_df['TransactionType'] == 'S-Sale']

,TickerId,ReportingCik,CompanyCik,TransactionType,SecuritiesOwned,SecuritiesTransacted,ReportingName,TypeOfOwner,SecurityName,Price,FormType,AcquisitionOrDisposition,TransactionDate,FilingDate
446663,3630,0001198053,0000713676,S-Sale,253615.0,100.0,DEMCHAK WILLIAM S,officer: Vice Chairman,$5 Par Common Stock,70.2900,4,D,2006-11-29,2006-12-01 16:07:17
447160,3630,0001198053,0000713676,S-Sale,490847.0,165442.0,DEMCHAK WILLIAM S,officer: President/CEO,$5 Par Common Stock,127.1028,4,D,2017-02-28,2017-03-02 13:58:09
447598,3630,0001198053,0000713676,S-Sale,221615.0,800.0,DEMCHAK WILLIAM S,officer: Vice Chairman,$5 Par Common Stock,70.6600,4,D,2006-11-29,2006-12-01 16:09:00
447617,3630,0001198053,0000713676,S-Sale,248251.0,700.0,DEMCHAK WILLIAM S,officer: Vice Chairman,$5 Par Common Stock,75.5300,4,D,2007-02-20,2007-02-22 16:12:04
448010,3630,0001198053,0000713676,S-Sale,189951.0,8600.0,DEMCHAK WILLIAM S,officer: Vice Chairman,$5 Par Common Stock,75.6100,4,D,2007-02-15,2007-02-16 16:25:25
448144,3630,0001198053,0000713676,S-Sale,190953.0,63094.0,DEMCHAK WILLIAM S,officer: Vice Chairman,$5 Par Common Stock,71.9374,4/A,D,2008-07-31,2008-08-04 16:18:51
448187,3630,0001198053,0000713676,S-Sale,191023.0,63024.0,DEMCHAK WILLIAM S,officer: Vice Chairman,$5 Par Common Stock,71.9374,4,D,2008-07-31,2008-08-01 16:17:57
448478,3630,0001198053,0000713676,S-Sale,225951.0,600.0,DEMCHAK WILLIAM S,officer: Vice Chairman,$5 Par Common Stock,75.9000,4,D,2007-02-20,2007-02-22 16:17:54
448603,3630,0001198053,0000713676,S-Sale,258762.0,34900.0,DEMCHAK WILLIAM S,officer: Senior Vice Chairman,$5 Par Common Stock,55.5200,4,D,2011-12-05,2011-12-07 17:34:54
448988,3630,0001198053,0000713676,S-Sale,256207.0,56584.0,DEMCHAK WILLIAM S,officer: President/CEO,$5 Par Common Stock,75.4357,4,D,2013-10-23,2013-10-25 16:27:13


In [102]:
ticker_df

,TickerId,ReportingCik,CompanyCik,TransactionType,SecuritiesOwned,SecuritiesTransacted,ReportingName,TypeOfOwner,SecurityName,Price,FormType,AcquisitionOrDisposition,TransactionDate,FilingDate
446556,3630,0001198053,0000713676,J-Other,40889.0,28171.0,DEMCHAK WILLIAM S,officer: Vice Chairman and CFO,Phantom Stock Unit,0.00,4,A,2004-04-26,2004-04-28 16:08:45
446565,3630,0001198053,0000713676,P-Purchase,110110.0,4600.0,DEMCHAK WILLIAM S,officer: Vice Chairman and CFO,$5 Par Common Stock,46.69,4,A,2004-07-23,2004-07-26 13:15:15
446586,3630,0001198053,0000713676,J-Other,42212.0,389.0,DEMCHAK WILLIAM S,officer: Vice Chairman and CFO,Phantom Stock Unit,0.00,4,A,2005-01-24,2005-02-18 15:32:29
446599,3630,0001198053,0000713676,J-Other,1581.0,23.0,DEMCHAK WILLIAM S,officer: Vice Chairman and CFO,Phantom Stock Unit,0.00,4,A,2005-06-30,2005-07-05 11:08:00
446609,3630,0001198053,0000713676,J-Other,1764.0,24.0,DEMCHAK WILLIAM S,officer: Vice Chairman,Phantom Stock Unit,0.00,4,A,2005-10-06,2005-10-07 14:54:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1873840,3630,0001198053,0000713676,S-Sale,549733.0,1242.0,DEMCHAK WILLIAM S,"director, officer: CEO",$5 Par Common Stock,183.74,4,D,2024-09-27,2024-09-30 17:00:30
1877105,3630,0001198053,0000713676,S-Sale,548491.0,1242.0,DEMCHAK WILLIAM S,"director, officer: CEO",$5 Par Common Stock,181.35,4,D,2024-10-04,2024-10-07 14:50:30
1878348,3630,0001198053,0000713676,S-Sale,547249.0,1242.0,DEMCHAK WILLIAM S,"director, officer: CEO",$5 Par Common Stock,185.00,4,D,2024-10-11,2024-10-15 14:25:52
1879549,3630,0001198053,0000713676,S-Sale,546007.0,1242.0,DEMCHAK WILLIAM S,"director, officer: CEO",$5 Par Common Stock,190.57,4,D,2024-10-18,2024-10-21 13:45:44
